<a href="https://colab.research.google.com/github/Indukurivigneshvarma/Deep_Learning/blob/main/NLP/GPT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets --quiet

from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [3]:
dataset = load_dataset("imdb", split="train[:1%]")
dataset = dataset.train_test_split(test_size=0.1)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [7]:
def tokenize_function(examples):
    outputs = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/225 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [8]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [9]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-imdb",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to=[]
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)

In [11]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.795300,3.340366


TrainOutput(global_step=113, training_loss=3.7340621779450274, metrics={'train_runtime': 28.3494, 'train_samples_per_second': 7.937, 'train_steps_per_second': 3.986, 'total_flos': 14697676800000.0, 'train_loss': 3.7340621779450274, 'epoch': 1.0})

In [12]:
prompt = "the movie was great with"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_length=80,
    num_return_sequences=3,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    do_sample=True
)

for i, output in enumerate(outputs):
    print(f"\n=== Generated {i+1} ===\n")
    print(tokenizer.decode(output, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== Generated 1 ===

the movie was great with the acting, it's not the best movie of the decade in the history of the genre. The acting, the humor, the musicality, the musicality are the worst in the whole movie. The plot of this movie is stupid, it's not funny, and I don't see any way to make this movie better. The acting is not the most creative and the

=== Generated 2 ===

the movie was great with very much of the plot but in the end there is more of a plot than a lot of it. I was excited to see what the plot had to do with the story which could have been about a gang of assassins. I really enjoyed the movie, the acting and the acting is excellent and the plot was very interesting. The acting was quite good and it was quite simple

=== Generated 3 ===

the movie was great with the dialogue. Some things that I liked about it are the script. There wasn't much to say in the movie. There is no plot to the film. Nothing. The character development was a mess. The dialogue was dull in th

In [13]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True \
    --to notebook your_notebook.ipynb --output your_notebook_clean.ipynb


[NbConvertApp] WARNING | pattern 'your_notebook.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
 